In [ ]:
#!conda env update --file environment.yml --prune

Using sorted batching, e.g:
each librispeech wav will be treated separately to others in its same parent folder.
wavs will be sorted by duration

In [13]:
import os, random, glob, ntpath
import numpy as np

import torch, torchaudio
import torchaudio.functional as F
from IPython.display import Audio

from tqdm import tqdm

mode = "train"

speech_list = glob.glob("data/speech/" + mode + "**/*.wav",recursive=True)
noise_list = glob.glob("data/noise/" + mode + "**/*.wav",recursive=True)

Basic

In [ ]:
for speech_file in speech_list:
	snr = random.randint(-1,2) * 5
	noise_file = random.choice(noise_list)	

	speech_name = ntpath.basename(speech_file).removesuffix(".wav")
	noise_name = ntpath.basename(noise_file).removesuffix(".wav")
	output_path = f"{"data/mixed/"+mode}/{speech_name}_{noise_name}_{snr}.wav"

	speech,_ = torchaudio.load("data/speech/train/test.wav")
	noise,_ = torchaudio.load("data/noise/train/ch01.wav")
	noise = noise[:,: speech.shape[1]]
	snr = torch.tensor(snr)
	mixed = F.add_noise(speech, noise, snr)

	torchaudio.save(output_path,mixed,sample_rate=16000, format="wav", encoding="PCM_S")

Sorted

In [ ]:
max_samples = 100
is_descending = True
# rnd_seed = 42
rnd_seed = random.random()
rnd = random.Random(rnd_seed)

lst :list[list[int,int]] = []
speech_list = glob.glob("data/speech/" + mode + "/**/*.wav",recursive=True)
noise_list = glob.glob("data/noise/" + mode + "/**/*.wav",recursive=True)
print(len(speech_list))

for file in tqdm(speech_list):
	frames = torchaudio.info(file).num_frames
	lst.append([frames,file])

#Sort by num_frames, in descending order
lst.sort(key=lambda x: x[0], reverse=is_descending)

speech = 0
noise = 0
for _, file in tqdm(lst[0:1]):
	speech, _ = torchaudio.load(file, format="wav", normalize=False)
	noise_file = rnd.choice(noise_list)
	noise, _ = torchaudio.load(noise_file, format="wav", normalize=False)
	length_diff = noise.shape[1] - speech.shape[1]
	offset = rnd.randint(0,length_diff)
	noise = noise[:,offset:offset+speech.shape[1]]
	print(speech.shape)
	print(noise.shape)

Audio(speech.numpy()[0],rate=16000)
# Audio(noise.numpy()[0],rate=16000)






In [ ]:
print(lst[:10])

TypeError: list indices must be integers or slices, not tuple